# Import Libraries

In [1]:
%%capture
# Install important libraries
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
# Save the IAM role in a variable called `role`.
# This would be useful when training the model.
role = sagemaker.get_execution_role()

In [4]:
# The train and val paths below are public S3 buckets created by Udacity for
# this project
inputs = {
    'train': 's3://cd2688-object-detection-tf2/train/',
    'val': 's3://cd2688-object-detection-tf2/val/'
}

# Path for the S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://sdc-object-detection-bucket/logs/'

In [5]:
%%bash
# Create a checkpoint folder for all the models
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint

# Model Selection

Train multiple of-the-shelf models provided in Tensorflow Object detection API:

- CenterNet MobileNetV2 FPN 512x512
- EfficientDet D0 512x512
- SSD MobileNet v2 320x320
- Faster R-CNN ResNet50 V1 640x640

## CenterNet

In [6]:
%%bash
# Downlad the model
wget -O /tmp/centernet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz

--2023-12-31 22:20:42--  http://download.tensorflow.org/models/object_detection/tf2/20210210/centernet_mobilenetv2fpn_512x512_coco17_od.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.111.207, 142.251.16.207, 172.253.62.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.111.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44016454 (42M) [application/x-tar]
Saving to: ‘/tmp/centernet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 11.5M 4s
    50K .......... .......... .......... .......... ..........  0% 22.1M 3s
   100K .......... .......... .......... .......... ..........  0% 23.1M 2s
   150K .......... .......... .......... .......... ..........  0% 15.8M 2s
   200K .......... .......... .......... .......... ..........  0% 22.1M 2s
   250K .......... .......... .......... .......... ..........  0% 22.8M 2s
   300K .......... .......... .......... .......... .........

In [7]:
%%bash
# Extract checkpiont folder
tar -zxvf /tmp/centernet.tar.gz --strip-components 2 --directory source_dir/models/centernet/checkpoint centernet_mobilenetv2_fpn_od/checkpoint

centernet_mobilenetv2_fpn_od/checkpoint/ckpt-301.index
centernet_mobilenetv2_fpn_od/checkpoint/checkpoint
centernet_mobilenetv2_fpn_od/checkpoint/ckpt-301.data-00000-of-00001
